In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/beyond-analysis/train.csv
/kaggle/input/beyond-analysis/sample_submission_random.csv
/kaggle/input/beyond-analysis/test.csv


In [2]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt

In [3]:
train_dataset = pd.read_csv('/kaggle/input/beyond-analysis/train.csv')
train_dataset.describe()

,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,...,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2
count,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,...,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06
mean,9.852168e+10,1.105455e+01,4.223696e-01,9.868451e-01,4.726720e+00,5.350111e-01,3.700804e+00,4.262171e-02,4.482682e-01,7.602949e-01,...,1.604540e-03,1.379019e-01,2.268259e+00,7.415130e-01,3.447181e-01,2.749940e-01,3.990834e-02,1.652854e-02,6.508164e+00,2.081873e+02
std,6.339280e+06,7.850787e+00,4.939370e-01,1.139379e-01,2.609701e+01,2.557525e+00,2.222386e+01,8.355851e-01,2.193891e+00,3.407900e+00,...,1.532719e-02,1.171732e+00,4.263824e+00,1.567570e+00,5.121462e+00,4.754888e+00,4.349036e-01,1.926375e-01,1.162251e+01,2.269953e+02
min,9.848127e+10,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.604128e+00,0.000000e+00
25%,9.851986e+10,4.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.012769e+00,3.835616e+01
50%,9.852279e+10,1.000000e+01,0.000000e+00,1.000000e+00,4.500000e-01,5.970000e-02,1.888000e-01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,7.000000e-01,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.202048e+00,1.150685e+02
75%,9.852542e+10,1.700000e+01,1.000000e+00,1.000000e+00,1.962600e+00,2.637500e-01,1.276900e+00,0.000000e+00,0.000000e+00,5.000000e-01,...,0.000000e+00,0.000000e+00,2.300000e+00,8.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.042758e+00,3.068493e+02
max,9.852732e+10,3.000000e+01,1.000000e+00,1.000000e+00,2.819799e+03,2.096527e+02,2.086580e+03,3.531417e+02,2.000000e+02,1.250000e+02,...,1.300000e+00,6.450000e+02,1.077000e+02,4.510000e+01,6.140000e+02,6.732000e+02,3.330000e+01,1.870000e+01,8.340632e+02,1.000000e+03


In [4]:
train_dataset

,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,CATEGORY_1,CATEGORY_2,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,...,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2
0,98481267304,1,0,M,B,1,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449287,101.369863
1,98481267304,2,0,M,B,1,0.137350,0.011550,0.09990,0.0,...,0.0,0.0,1.8,1.0,0.0,0.0,0.0,0.0,4.449287,101.369863
2,98481267304,3,0,M,B,1,0.158350,0.010425,0.02420,0.0,...,0.0,0.0,0.7,0.3,0.0,0.0,0.0,0.0,4.449287,101.369863
3,98481267304,4,0,M,B,1,0.444900,0.035850,0.23980,0.0,...,0.0,0.0,2.8,1.5,0.0,0.0,0.0,0.0,4.449287,101.369863
4,98481267304,5,0,M,B,1,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449287,101.369863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246832,98527316143,30,0,B,D,1,0.100000,0.015000,0.23375,0.0,...,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,4.086504,156.164384
1246833,98527316168,1,0,B,D,1,0.370533,0.048900,0.12320,0.0,...,0.0,0.0,0.5,0.1,0.0,0.0,0.0,0.0,3.959112,43.835616
1246834,98527316168,2,0,B,D,1,0.371137,0.040704,0.09790,0.0,...,0.0,0.0,2.5,0.2,0.0,0.0,0.0,0.0,3.959112,43.835616
1246835,98527316168,3,0,B,D,1,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.959112,43.835616


In [5]:
t_df = train_dataset
t_df = t_df.groupby('UNIQUE_IDENTIFIER')
t_df = t_df.agg(np.mean)
t_df

,SEQUENCE_NO,STATUS_CHECK,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,...,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2
UNIQUE_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,
98481267304,4.0,0.0,1.000000,0.112236,0.008682,0.052929,0.000000,0.000000,0.035714,0.014286,...,0.000000,0.000000,0.928571,0.442857,0.000000,0.0,0.000000,0.0,4.449287,101.369863
98481267698,10.0,1.0,0.947368,30.285742,3.733455,25.120618,0.000000,1.163158,10.263158,0.052632,...,0.010526,0.684211,0.700000,0.247368,0.000000,0.0,0.000000,0.0,7.182779,115.068493
98481269325,9.5,0.0,1.000000,0.083720,0.007540,0.052828,0.000000,0.033333,0.013889,0.005556,...,0.000000,0.027778,1.311111,0.305556,0.000000,0.0,0.000000,0.0,3.977371,24.657534
98481271512,3.0,0.0,1.000000,0.777000,0.122220,0.017000,0.000000,0.240000,0.700000,0.040000,...,0.000000,0.540000,0.900000,0.040000,0.000000,0.0,0.000000,0.0,3.988122,10.958904
98481273023,1.0,0.0,1.000000,0.500000,0.080000,0.000000,0.000000,0.000000,0.500000,0.100000,...,0.000000,0.000000,0.200000,0.000000,0.000000,0.0,0.000000,0.0,4.128875,46.575342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98527316109,2.5,0.0,1.000000,12.689407,1.655539,7.196875,0.000000,9.300000,2.875000,0.125000,...,0.000000,0.000000,2.875000,0.725000,0.000000,0.0,0.000000,0.0,3.876182,10.958904
98527316131,5.0,0.0,1.000000,38.264003,4.534993,31.702444,0.000000,1.044444,4.055556,0.122222,...,0.000000,1.055556,11.811111,3.188889,0.033333,0.0,0.011111,0.0,5.516531,24.657534
98527316133,15.5,0.0,1.000000,0.998121,0.133820,0.650847,0.280823,0.040000,0.325000,0.053333,...,0.000000,0.003333,1.096667,0.243333,0.000000,0.0,0.000000,0.0,5.268355,449.315068


In [6]:
# No missing values, 2 categorical cols = ['CATEGORY_1', 'CATEGORY_2']
# ['SEQUENCE_NO', 'DEPOSIT_2', 'DEPOSIT_TRAILS', 'CATEGORY_1', 'CATEGORY_2']
# Split into userwise means/sums

def cleanse(df):
    df = df.groupby('UNIQUE_IDENTIFIER')
    df = df.agg(np.mean)
    to_drop = ['DEPOSIT_2', 'DEPOSIT_TRAILS']
#     , 'CATEGORY_1', 'CATEGORY_2', 'UNIQUE_IDENTIFIER'
    return df.drop(to_drop, axis=1)

T_df = train_dataset
T_df = cleanse(T_df)

X = T_df.drop(['Y1', 'Y2'], axis=1)
y = T_df[['Y1', 'Y2']]

# X = cleanse(X)
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

(72223, 18) (72223, 2) (24075, 18) (24075, 2)


In [7]:
model_param1 = XGBRegressor(n_estimators=1000, learning_rate=0.01, tree_method='hist')
model_param1.fit(X_train, y_train['Y1'], early_stopping_rounds=5, eval_set=[(X_valid, y_valid['Y1'])], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='hist', validate_parameters=1, verbosity=None)

In [8]:
model_param2 = XGBRegressor(n_estimators=1000, learning_rate=0.01, tree_method='hist')
model_param2.fit(X_train, y_train['Y2'], early_stopping_rounds=5, eval_set=[(X_valid, y_valid['Y2'])], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='hist', validate_parameters=1, verbosity=None)

In [9]:
y1, y2 = y_valid['Y1'], y_valid['Y2']
print(f'Y1 rmse: {sqrt(mean_squared_error(y1, model_param1.predict(X_valid)))}')
print(f'Y2 rmse: {sqrt(mean_squared_error(y2, model_param1.predict(X_valid)))}')
# Y1 rmse: 10.26070193311643
# Y2 rmse: 303.04928245745634

Y1 rmse: 6.509652610964662
Y2 rmse: 216.7012225511206


In [10]:
test_dataset = pd.read_csv('/kaggle/input/beyond-analysis/test.csv')
print(test_dataset.shape)

valid_test_dataset = cleanse(test_dataset)
print(valid_test_dataset.shape)

test_dataset.drop_duplicates(subset=['UNIQUE_IDENTIFIER'], inplace=True)
print(test_dataset.shape)

(979059, 23)
(65242, 18)
(65242, 23)


In [11]:
# print(test_dataset['UNIQUE_IDENTIFIER'])
output = pd.DataFrame({'UNIQUE_IDENTIFIER': test_dataset['UNIQUE_IDENTIFIER'], 'Y1': model_param1.predict(valid_test_dataset), 'Y2': model_param2.predict(valid_test_dataset)})
output.to_csv('submission.csv', index=False)

In [12]:
pd.DataFrame({'Cols': X.columns.to_list(), 'Weightage': model_param1.feature_importances_}).sort_values(by=['Weightage'], ascending=False)

,Cols,Weightage
4,REVENUE,0.203544
1,STATUS_CHECK,0.111189
0,SEQUENCE_NO,0.099419
7,DISCOUNT,0.082483
5,WINNINGS_1,0.075869
12,ENTRY_NUMBER,0.052730
16,PRACTICE_ENTRY_NUMBER,0.049902
9,DEPOSIT_NUMBER,0.046474
10,WITHDRAW,0.043333
8,DEPOSIT,0.032441


In [13]:
pd.DataFrame({'Cols': X.columns.to_list(), 'Weightage': model_param2.feature_importances_}).sort_values(by=['Weightage'], ascending=False)

,Cols,Weightage
0,SEQUENCE_NO,0.500452
1,STATUS_CHECK,0.257963
13,WINNINGS_NUMBER,0.037491
12,ENTRY_NUMBER,0.035032
9,DEPOSIT_NUMBER,0.024333
10,WITHDRAW,0.017731
8,DEPOSIT,0.016562
5,WINNINGS_1,0.014526
4,REVENUE,0.012837
6,WINNINGS_2,0.012583
